# Automatic Differentiation with `autograd`

We train models to get better and better as a function of experience. Usually, getting better means minimizing a loss function. To achieve this goal, we often iteratively compute the gradient of the loss with respect to weights and then update the weights accordingly. While the gradient calculations are straightforward through a chain rule, for complex models, working it out by hand can be a pain.

Before diving deep into the model training, let's go through how MXNet’s `autograd` package expedites this work by automatically calculating derivatives.

## Basic usage

Let's first import the `autograd` package.

In [1]:
from mxnet import nd
from mxnet import autograd

### Case Study: Autograd for $f(x) = 2 x^2$

Let's start by assigning an initial value of $x$.

In [2]:
x = nd.array([[1, 2], [3, 4]])
x


[[1. 2.]
 [3. 4.]]
<NDArray 2x2 @cpu(0)>

#### Attach Gradient Storage

Calculating gradients require extra computation, and we’ll need a place to store it.

In MXNet, we can tell an NDArray that we plan to calculate and store a gradient by invoking its `attach_grad` method.

In [3]:
x.attach_grad()

#### Define and Record y = f(x)

Now we’re going to define the function $y=f(x)$. To let MXNet store $y$, so that we can compute gradients later, we need to put the definition inside a `autograd.record()` scope.

In [4]:
with autograd.record():
    y = 2 * x**2
print(y)


[[ 2.  8.]
 [18. 32.]]
<NDArray 2x2 @cpu(0)>


#### Invoke Back Propagation

Let’s invoke back propagation (backprop) by calling `y.backward()`.

In [5]:
y.backward()

#### Verify Computed Gradients

Note that $y=2x^2$ and $\frac{dy}{dx} = 4x$, which should be

`[[4, 8],[12, 16]]`

In [6]:
x.grad


[[ 4.  8.]
 [12. 16.]]
<NDArray 2x2 @cpu(0)>

## Using Python Control Flows

Sometimes we want to write dynamic programs where the execution depends on some real-time values. MXNet will record the execution trace and compute the gradient as well.

### Cast Study: 

`f(a)`: it doubles `a` until `norm(a)` reaches 1000. Then it selects one element depending on the sum of its elements.

In [7]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    return b[0] if b.sum() >= 0 else b[1]

#### Feed in a Random Value and Record:

In [8]:
a = nd.random.uniform(shape=2)
a.attach_grad()
with autograd.record():
    c = f(a)

#### Compute and Verify Gradients

`b` is a linear function of `a`, and `c` is chosen from `b`. 

The gradient w.r.t. `a` be will be either `[c/a[0], 0]` or `[0, c/a[1]]`.

```python
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    return b[0] if b.sum() >= 0 else b[1]
```

In [9]:
c.backward()
[a.grad, c/a]

[
 [2048.    0.]
 <NDArray 2 @cpu(0)>, 
 [2048.     1895.8933]
 <NDArray 2 @cpu(0)>]